### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Endometriosis/GSE165004'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_age = None  # define the functions when applicable

# Check for gene expression data availability
series_title = "!Series_title\t'Endometrial Tissue RNA expression in recurrent pregnancy losses and unexplained infertility vs. control'"
is_gene_available = "RNA expression" in series_title

# Check for the availability and key of relevant variables in the sample characteristics dictionary
sample_characteristics_dict = {
    0: ['subject status/group: Control', 'subject status/group: patient with RPL', 'subject status/group: patient with UIF'],
    1: ['tissue: Endometrial tissue']
}

# Checking the availability of 'Endometriosis'
if 0 in sample_characteristics_dict and len(set(sample_characteristics_dict[0])) > 1:
    trait_row = 0

# 'age' and 'gender' are not explicitly mentioned in the sample characteristics, so they are set to None
age_row = None
gender_row = None

# Define a function for converting the 'Endometriosis' variable
def convert_trait(value):
    mapping = {
        'Control': 0,
        'patient with RPL': 1,
        'patient with UIF': None,  # "UIF" here stands for "unexplained infertility," hence not Endometriosis
    }
    key_value = value.split(':')[-1].strip()
    return mapping.get(key_value, None)

# Since there is no 'age' and 'gender' data, define `convert_age` and `convert_gender` as pass-through functions returning None
def convert_age(value):
    return None

def convert_gender(value):
    return None

# Save cohort information
save_cohort_info('GSE165004', './preprocessed/Endometriosis/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction (retained in case clinical data becomes available in further steps)
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Endometriosis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Endometriosis/trait_data/GSE165004.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:

# 1. Identify the keys for identifiers and gene symbols from the gene annotation data
identifier_key = 'NAME'
gene_symbol_key = 'GENE_SYMBOL'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)
mapping_df = mapping_df.rename(columns={identifier_key: 'ID'})

# 3. Ensure 'ID' is properly set in `gene_data` to align with `apply_gene_mapping`
gene_data.index.name = 'ID'

# 4. Apply the mapping and name the resulting gene expression dataframe "gene_data"
gene_data = apply_gene_mapping(gene_data, mapping_df)

print("Preview of mapped gene data:")
print(preview_df(gene_data))


### Step 7: Data Normalization and Merging

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Endometriosis/gene_data/GSE165004.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data are severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Endometriosis')

# 4. Save the cohort information.
save_cohort_info('GSE165004', './preprocessed/Endometriosis/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Endometriosis/GSE165004.csv'
    unbiased_merged_data.to_csv(csv_path)
